## Book Processing

This Jupyter notebook processes raw books csv data performing basic data cleaning:

1. Selecting only desired features (raw/derived)
2. String cleaning
3. Handling missing values

#### Import Libraries

In [10]:
import pandas as pd

#### Data Upload

In [11]:
path = "C:\\Users\\juhic\\OneDrive\\Desktop\\goodreads_kaggle_books.csv"
books = pd.read_csv(path, usecols = ['title',                       
                                   'series',
                                   'author',
                                   'rating_count',
                                   'review_count',
                                   'average_rating',
                                   'five_star_ratings',
                                   'four_star_ratings',
                                   'three_star_ratings',
                                   'two_star_ratings',
                                   'one_star_ratings',
                                   'number_of_pages',
                                   'date_published',
                                   'publisher',
                                   'genre_and_votes'])


# Rename columns ------------------------------------------------------
cols = {'number_of_pages': 'page_count',
        'date_published': 'published_year',
        'genre_and_votes': 'genre'}
books.rename(columns = cols, inplace = True)

#### Data Overview I

In [12]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52199 entries, 0 to 52198
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               52199 non-null  object 
 1   series              23319 non-null  object 
 2   author              52199 non-null  object 
 3   rating_count        52199 non-null  int64  
 4   review_count        52199 non-null  int64  
 5   average_rating      52199 non-null  float64
 6   five_star_ratings   52199 non-null  int64  
 7   four_star_ratings   52199 non-null  int64  
 8   three_star_ratings  52199 non-null  int64  
 9   two_star_ratings    52199 non-null  int64  
 10  one_star_ratings    52199 non-null  int64  
 11  page_count          49869 non-null  float64
 12  published_year      51339 non-null  object 
 13  publisher           48563 non-null  object 
 14  genre               49359 non-null  object 
dtypes: float64(2), int64(7), object(6)
memory usage: 6.0+

In [13]:
books.head()

,title,series,author,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,page_count,published_year,publisher,genre
0,Inner Circle,(Private #5),"Kate Brian, Julian Peploe",7597,196,4.03,3045,2323,1748,389,92,220.0,January 1st 2007,Simon Schuster Books for Young Readers,"Young Adult 161, Mystery 45, Romance 32"
1,A Time to Embrace,(Timeless Love #2),Karen Kingsbury,4179,177,4.35,2255,1290,518,93,23,400.0,October 29th 2006,Thomas Nelson,"Christian Fiction 114, Christian 45, Fiction 3..."
2,Take Two,(Above the Line #2),Karen Kingsbury,6288,218,4.23,3000,2020,1041,183,44,320.0,January 1st 2009,Zondervan,"Christian Fiction 174, Christian 81, Fiction 58"
3,Reliquary,(Pendergast #2),"Douglas Preston, Lincoln Child",38382,1424,4.01,12711,15407,8511,1429,324,464.0,1997,Tor Books,"Thriller 626, Mystery 493, Horror 432, Fiction..."
4,The Millionaire Next Door: The Surprising Secr...,NaN,"Thomas J. Stanley, William D. Danko",72168,3217,4.04,27594,25219,14855,3414,1086,258.0,October 28th 1995,Gallery Books,"Economics-Finance 1162, Nonfiction 910, Busine..."


#### String Cleaning

In [14]:
cols = ['title','series','author','publisher','genre']

# lowercase, strip spaces from ends
books[cols] = books[cols].apply(func = lambda x: x.str.lower().str.strip(), axis = 1)

# strip spaces in between
for c in cols:
    books[c] = books[c].str.replace(r" +", " ")

    
# Author: select single author
books['author'] = books['author'].str.split(',', expand = True)[0]

# Series: extract alphabetical characters
books['series'] = books['series'].str.lstrip('(').str.rstrip(')').str.split('#', expand = True)[0]

# volume flag: if the book is part of a series ---
books['series'] = books['series'].fillna('')
books['is_volume'] = ['no' if i == '' else 'yes' for i in books['series']]

# Genre: select most voted genre, drop records with null
books['genre'] = books['genre'].str.split(',', expand = True)[0].str.rsplit(' ', 1, expand = True)[0]

# Published Date: extract year
books['published_year'] = books['published_year'].str.extract(r'(\d{4})')

<ipython-input-14-1725cd5b2eb0>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  books[c] = books[c].str.replace(r" +", " ")


#### Handling Missing values

In [15]:
print(f"Null records \n")
for c in books.columns:
    i = books[books[c].isnull()].shape[0]
    print(f"{c}: {i}")

Null records 

title: 0
series: 0
author: 0
rating_count: 0
review_count: 0
average_rating: 0
five_star_ratings: 0
four_star_ratings: 0
three_star_ratings: 0
two_star_ratings: 0
one_star_ratings: 0
page_count: 2330
published_year: 1009
publisher: 3636
genre: 2840
is_volume: 0


In [16]:
# Drop records with null values (except from publisher):

books.drop(axis = 0, index = books[books['genre'].isnull()].index, inplace = True)
books.drop(axis = 0, index = books[books['page_count'].isnull()].index, inplace = True)
books.drop(axis = 0, index = books[books['published_year'].isnull()].index, inplace = True)
books.reset_index(drop = True, inplace = True)

In [17]:
# Drop publisher as it is not required in analysis:

books.drop(axis = 1, columns = 'publisher', inplace = True)

#### Data Overview II

In [18]:
books.head()

,title,series,author,rating_count,review_count,average_rating,five_star_ratings,four_star_ratings,three_star_ratings,two_star_ratings,one_star_ratings,page_count,published_year,genre,is_volume
0,inner circle,private,kate brian,7597,196,4.03,3045,2323,1748,389,92,220.0,2007,young adult,yes
1,a time to embrace,timeless love,karen kingsbury,4179,177,4.35,2255,1290,518,93,23,400.0,2006,christian fiction,yes
2,take two,above the line,karen kingsbury,6288,218,4.23,3000,2020,1041,183,44,320.0,2009,christian fiction,yes
3,reliquary,pendergast,douglas preston,38382,1424,4.01,12711,15407,8511,1429,324,464.0,1997,thriller,yes
4,the millionaire next door: the surprising secr...,,thomas j. stanley,72168,3217,4.04,27594,25219,14855,3414,1086,258.0,1995,economics-finance,no


In [19]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47213 entries, 0 to 47212
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               47213 non-null  object 
 1   series              47213 non-null  object 
 2   author              47213 non-null  object 
 3   rating_count        47213 non-null  int64  
 4   review_count        47213 non-null  int64  
 5   average_rating      47213 non-null  float64
 6   five_star_ratings   47213 non-null  int64  
 7   four_star_ratings   47213 non-null  int64  
 8   three_star_ratings  47213 non-null  int64  
 9   two_star_ratings    47213 non-null  int64  
 10  one_star_ratings    47213 non-null  int64  
 11  page_count          47213 non-null  float64
 12  published_year      47213 non-null  object 
 13  genre               47213 non-null  object 
 14  is_volume           47213 non-null  object 
dtypes: float64(2), int64(7), object(6)
memory usage: 5.4+

#### Data Download I

In [20]:
# Download intermediate processed books data ---------------------------------------
books.to_csv('books_processed.csv')